# File Suggestions

In this lesson, you will design an agent that can read the user goal, then use tools to evaluate
available files to suggest relevant data sources.

You'll learn:
- how memory helps coordinate agent tasks
- how tools can be be used to access the environment
- how to trust, but verify inside of tools


<img src="/Users/rajesh/Desktop/rajesh/Archive/teaching/agentic_ai/BMGR-MAY2025-GIAI-2/multiagent_with_graph_datastructure/googleADK/images/entire_solution.png" width="500">

The File Suggestion agent is a tool-use agent that suggests files to use for import, based on the approved user goal from the previous lesson.


- Input: `approved_user_goal`, a dictionary pairing a kind of graph with a description of the purpose of the graph.
- Output: `approved_files`, a list of files that have been approved for import.
- Tools: `get_approved_user_goal`, `list_import_files`, `sample_file`, `set_suggested_files`, `approve_suggested_files`

In [15]:
# Import necessary libraries
import os
from pathlib import Path

from itertools import islice

from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import ToolContext
from google.genai import types # For creating message Content/Parts

# For type hints
from typing import Dict, Any, List

# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb, tool_success, tool_error

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


In [16]:
# --- Define Model Constants for easier use ---
MODEL_GPT_4O = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT_4O)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, messages=[{"role": "user", "content": "Are you ready?"}], tools=[]))

print("\nOpenAI is ready.")


ModelResponse(id='chatcmpl-CCiBhQ4gL8zrUh7PwjMu7MCGx0ZvO', created=1757145829, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_cbf1785567', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Yes, I'm ready. How can I assist you today?", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=13, prompt_tokens=27, total_tokens=40, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

OpenAI is ready.


# Define the File Suggestion Agent

In [33]:
file_suggestion_agent_instruction = """
You are a constructive critic AI reviewing a list of files. Your goal is to suggest relevant files
for constructing a knowledge graph.

**Task:**
Review the file list for relevance to the kind of graph and description specified in the approved user goal. 

For any file that you're not sure about, use the 'sample_file' tool to get 
a better understanding of the file contents. 

Only consider structured data files like CSV or JSON.

Prepare for the task:
- use the 'get_approved_user_goal' tool to get the approved user goal

Think carefully, repeating these steps until finished:
1. list available files using the 'list_available_files' tool
2. evaluate the relevance of each file, then record the list of suggested files using the 'set_suggested_files' tool
3. use the 'get_suggested_files' tool to get the list of suggested files
4. ask the user to approve the set of suggested files
5. If the user has feedback, go back to step 1 with that feedback in mind
6. If approved, use the 'approve_suggested_files' tool to record the approval
"""

In [ ]:
# sample_file -- Done 
# get_approved_user_goal. - Done
# set_suggested_files 
# get_suggested_files
# approve_suggested_files
# list_available_files


In [ ]:
# d = {"a": 1, "b": 3}

In [ ]:
# d.get("a")

1

In [ ]:
# d["a"]

1

In [ ]:
# def devide(a, b):
#     try:
#         c = a/b
#         print(c)
#     except Exception as e:
#         print(str(e)) 
#     print("I am learning Google ADk")




In [17]:
# devide(2,0)

### Tool Definitions

In [26]:
# import tools defined in previous lesson
from tools import get_approved_user_goal, sample_file
from helper import get_neo4j_import_dir

In [ ]:
# sample_file -- Done 
# get_approved_user_goal. - Done
# set_suggested_files   - Done
# get_suggested_files - Done
# approve_suggested_files 
# list_available_files.  - Done


In [27]:
# this constant will be used as the key for storing the file list in the tool context state
ALL_AVAILABLE_FILES = "all_available_files"

def list_available_files(tool_context:ToolContext) -> dict:
    f"""Lists files available for knowledge graph construction.
    All files are relative to the import directory.

    Returns:
        dict: A dictionary containing metadata about the content.
                Includes a 'status' key ('success' or 'error').
                If 'success', includes a {ALL_AVAILABLE_FILES} key with list of file names.
                If 'error', includes an 'error_message' key.
                The 'error_message' may have instructions about how to handle the error.
    """
    # get the import dir using the helper function
    import_dir = Path(get_neo4j_import_dir())
    file_names = [str(file.relative_to(import_dir)) for file in import_dir.rglob("*") if file.is_file()]
    # save the list to state so we can inspect it later
    tool_context.state[ALL_AVAILABLE_FILES] = file_names
    return tool_success(ALL_AVAILABLE_FILES, file_names)


In [28]:
# Tool: Set/Get suggested files
SUGGESTED_FILES = "suggested_files"

def set_suggested_files(suggest_files:List[str], tool_context:ToolContext) -> Dict[str, Any]:
    """Set the suggested files to be used for data import.

    Args:
        suggest_files (List[str]): List of file paths to suggest

    Returns:
        Dict[str, Any]: A dictionary containing metadata about the content.
                Includes a 'status' key ('success' or 'error').
                If 'success', includes a {SUGGESTED_FILES} key with list of file names.
                If 'error', includes an 'error_message' key.
                The 'error_message' may have instructions about how to handle the error.
    """
    tool_context.state[SUGGESTED_FILES] = suggest_files
    return tool_success(SUGGESTED_FILES, suggest_files)


# Helps encourage the LLM to first set the suggested files.
# This is an important strategy for maintaining consistency through defined values.
def get_suggested_files(tool_context:ToolContext) -> Dict[str, Any]:
    """Get the files to be used for data import.

    Returns:
        Dict[str, Any]: A dictionary containing metadata about the content.
                Includes a 'status' key ('success' or 'error').
                If 'success', includes a {SUGGESTED_FILES} key with list of file names.
                If 'error', includes an 'error_message' key.
    """
    return tool_success(SUGGESTED_FILES, tool_context.state[SUGGESTED_FILES])

In [29]:
# Tool: Approve Suggested Files
# Just like the previous lesson, you'll define a tool which
# accepts no arguments and can sanity check before approving.
APPROVED_FILES = "approved_files"

def approve_suggested_files(tool_context:ToolContext) -> Dict[str, Any]:
    """Approves the {SUGGESTED_FILES} in state for further processing as {APPROVED_FILES}.
    
    If {SUGGESTED_FILES} is not in state, return an error.
    """
    if SUGGESTED_FILES not in tool_context.state:
        return tool_error("Current files have not been set. Take no action other than to inform user.")

    tool_context.state[APPROVED_FILES] = tool_context.state[SUGGESTED_FILES]
    return tool_success(APPROVED_FILES, tool_context.state[APPROVED_FILES])

In [30]:
# List of tools for the file suggestion agent
file_suggestion_agent_tools = [get_approved_user_goal, list_available_files, sample_file, 
    set_suggested_files, get_suggested_files,
    approve_suggested_files
]

### Agent Definition

In [34]:
file_suggestion_agent = Agent(
    name="file_suggestion_agent_v1",
    model = llm,
    description="Helps the user select files to import.",
    instruction=file_suggestion_agent_instruction,
    tools= file_suggestion_agent_tools
)

print(f"Agent '{file_suggestion_agent.name}' created.")

Agent 'file_suggestion_agent_v1' created.


### Interact with the Agent

In [36]:
from helper import make_agent_caller

file_suggestion_caller = await make_agent_caller(file_suggestion_agent,{
    "approved_user_goal":{
        "kind_of_graph": "supply chain analysis",
        "description": "A multi-level bill of materials for manufactured products, useful for root cause analysis.."
    }
})


In [37]:
# Run the Initial Conversation

# nudge the agent to look for files. in the full system, this will be the natural next step
await file_suggestion_caller.call("What files can we use for import?")


session_end = await file_suggestion_caller.get_session()

# expect that the agent has listed available files
print("Available files: ", session_end.state[ALL_AVAILABLE_FILES])

# the suggested files should be reasonable looking CSV files
print("Suggested files: ", session_end.state[SUGGESTED_FILES])



>>> User Query: What files can we use for import?
<<< Agent Response: I have suggested the following files for your supply chain analysis graph:

- `components.csv`
- `parts.csv`
- `suppliers.csv`
- `assemblies.csv`
- `part_supplier_mapping.csv`
- `product.csv`

Please review and let me know if you approve these suggestions or if any modifications are needed.
Available files:  ['components.csv', 'parts.csv', 'suppliers.csv', '.gitignore', 'assemblies.csv', 'part_supplier_mapping.csv', 'product.csv']
Suggested files:  ['components.csv', 'parts.csv', 'suppliers.csv', 'assemblies.csv', 'part_supplier_mapping.csv', 'product.csv']
